# Intro to PyTorch

[PyTorch](https://pytorch.org/tutorials/) is a leading open-source deep learning framework developed by Meta AI. It's favored by researchers and developers across various machine learning domains, including image recognition, NLP, and reinforcement learning, thanks to its flexibility and ease of use. Its dynamic nature allows for agile experimentation and debugging, catering to both beginners and experts in deep learning.

This notebook will be a short introduction to using PyTorch on a simple computer vision task - figuring out the digit in an image using the [MNIST Digit Dataset](https://en.wikipedia.org/wiki/MNIST_database).

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np

## Loading Data

PyTorch defines two major objects for loading data from file:
- [`Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) represents a collection of data to use in training deep learning models.  These work similar to a Python list, but can be customized extensively for better performance.
- [`DataLoader`](https://pytorch.org/docs/stable/data.html?highlight=data+loader#torch.utils.data.DataLoader) prepares data into batches for training neural networks (using mini-batch stochastic gradient descent), wrapping an input `Dataset` object.

In the example below, we use the `datasets` module to create two data loaders - one for our training set and one for our testing set.

In [ ]:
train_data = datasets.MNIST(
    root="private",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="private",
    train=False,
    download=True,
    transform=ToTensor()
)

In PyTorch (especially later on in this project), you can also create a custom dataset object for your own datasets - check out [this](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files) link on how to do so.

You can iterate through a dataset (like a Python list) to get a sense for the images. The data loader supports indexing as well.

In [ ]:
index = 0
for image, label in train_data:
    print(f"Image {index} has shape {image.shape}, corresponding to digit {label}")
    index += 1
    if index == 5:
        break

In [ ]:
plt.imshow(train_data[0][0].view(-1,28)) # have to do some reshaping to visualize the image properly

**Checkpoint**: Use `plt.imshow` and the `train_dataset` object to visualize the 30th image in the dataset in the cell below.

In [ ]:
# TODO: visualize the 30th image in the dataset

You can use the `DataLoader` object to construct a data loader around a dataset.

In [ ]:
train_loader = DataLoader(train_data, batch_size=32) # batch into 32 images / labels at a time
test_loader = DataLoader(test_data, batch_size=32) # batch into 32 images / labels at a time

index = 0
for image, label in train_loader:
    print(f"Batch {index} has shape {image.shape}, with corresponding labels of shape {label.shape}")
    index += 1
    if index == 5:
        break

## `nn.Module`

PyTorch defines the special `nn.Module` class to represent an arbitary neural network! Here is an example below:

In [ ]:
class Net(nn.Module): # Net inherits from nn.Module
    def __init__(self):
        """Constructor for the neural network."""
        super(Net, self).__init__()        # Call superclass constructor
        self.fc1 = nn.Linear(28 * 28, 128) # Create fully connected layer as an instance variable of Net 
        self.fc2 = nn.Linear(128, 10)      # Create another fully connected layer. Output = 10 for 10 classes
        self.relu = nn.ReLU()              # Activation function for this neural network
        self.flatten = nn.Flatten()        # Convert image to flat array

    def forward(self, x):
        """Forward pass for a neural network - predicts labels from input image"""
        x = self.flatten(x)
        x = self.relu(self.fc1(x)) # call the layers like functors to process inputs
        x = self.relu(self.fc2(x))
        return x

For a neural net to be valid for PyTorch it must:
- Inherit from `nn.Module` and call the superclass constructor using `super(Net, self).__init__()`
- Override the `forward` function and specify how to get predicted labels for some input image

You can create a neural network by calling its constructor

In [ ]:
model = Net()

**Checkpoint**: Add a second fully connected layer to the following neural network that has an input of 128 dimensions and an output of 32 dimensions.

In [ ]:
class CheckpointNet(nn.Module): 
    def __init__(self):
        """Constructor for the neural network."""
        super(Net, self).__init__()       
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = None # TODO: add a new fully connected layer
        self.fc2 = nn.Linear(32, 10)      
        self.relu = nn.ReLU()        
        self.flatten = nn.Flatten()      

    def forward(self, x):
        """Forward pass for a neural network - predicts labels from input image"""
        x = self.flatten(x)
        x = self.relu(self.fc1(x)) 
        x = None # TODO: Update the forward pass to include the new fully connected layer
        x = self.relu(self.fc3(x))
        return x

## Training Neural Network

To train any neural network, you need to specify the:
- loss function - in PyTorch, we can use Cross-Entropy Loss via `nn.CrossEntropyLoss`
- optimizer - algorithm for training model. We will use stochastic gradient descent (technically mini-batch SGD) via `torch.optim.SGD`

In [ ]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.007) # lr is the learning rate

To train the model, we use the following training loop - it is fairly common across PyTorch to use a similar training loop

In [ ]:
epochs = 20 # Number of epochs to train for
losses, accuracies = [], []

for epoch in range(epochs):
    model.train()

    for X, y in train_loader:
        optimizer.zero_grad()        # reset gradients
        outputs = model(X)           # make a prediction using the model
        loss = criterion(outputs, y) # compare predictions to ground truth labels
        loss.backward()              # calculate gradients
        optimizer.step()             # update parameters

    losses.append(loss.detach().item())

    model.eval()
    with torch.no_grad():
        # test performance after each epoch
        correct, total = 0, 0
        for X, y in test_loader:
            outputs = model(X)
            _, predicted = torch.max(outputs.data, 1) # get predicted digit
            total += len(y)
            correct += (predicted == y).sum().item()
        print(
            f"Epoch [{epoch+1}/{epochs}], Recent Loss: {loss.item():.4f}, Accuracy: {correct / total *100:.2f}%"
        )
        accuracies.append(correct / total)

We can visualize the training loss and test accuracy of our neural network change as the model trains:

In [ ]:
plt.plot(np.arange(len(losses)), losses)
plt.xlabel("Epoch")
plt.ylabel("Training Loss by Epoch")
plt.title("Training Loss")
plt.show()

**Checkpoint**: Use `plt.plot` and `accuracies` to visualize the test accuracy of the model as it trains

In [ ]:
# TODO: Visualize the test accuracy of model as it trains

## Model Testing

Let's test our model on the entire testing dataset now we're done training!

In [ ]:
model.eval()
with torch.no_grad():
    # test performance after each epoch
    correct, total = 0, 0
    for image, label in test_data:
        outputs = model(image.view(1, 1, 28, 28))
        _, predicted = torch.max(outputs.data, 1)  # get predicted digit
        total += 1
        correct += (predicted.item() == label)
    print(f"Accuracy: {correct / total *100:.2f}%")

Let's see some examples - consider the 5th image in the testing dataset. Our model predicts it to be the digit `4` - not bad!

In [ ]:
image3 = test_data[4][0]
plt.imshow(image3.view(-1, 28))
print(f"Predicted label: {model(image3).argmax()}, Actual label: {test_data[4][1]}")

Now consider the 2nd image in the testing dataset. Our model predicts it to be the digit `6`, when its actually `5`! It seems we have some more training to do ...

In [ ]:
image3 = test_data[8][0]
plt.imshow(image3.view(-1, 28))
print(f"Predicted label: {model(image3).argmax()}, Actual label: {test_data[8][1]}")

That's it! You've trained a model using PyTorch from scratch to be pretty good at classifying this dataset. Next week, we'll practice using **convolutional neural networks**!